In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt


df = pd.read_csv('nyc_open_space.csv')

df['geometry'] = df['the_geom'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

gdf = gdf[['PARK_NAME', 'LANDUSE', 'geometry']]

gdf_proj = gdf.to_crs(epsg=2263)

gdf_proj['centroid'] = gdf_proj.geometry.centroid

centroids_wgs84 = gdf_proj['centroid'].to_crs(epsg=4326)

gdf['centroid_lon'] = centroids_wgs84.x
gdf['centroid_lat'] = centroids_wgs84.y

gdf.to_file('nyc_open_space.geojson', driver='GeoJSON')

print("GeoJSON saved as nyc_open_space.geojson")


GeoJSON saved as nyc_open_space.geojson


In [ ]:
import geopandas as gpd

gdf = gpd.read_file('nyc_zip_codes.geojson')

gdf_proj = gdf.to_crs(epsg=2263)


gdf_proj['centroid'] = gdf_proj.geometry.centroid

centroids_wgs84 = gdf_proj['centroid'].to_crs(epsg=4326)

gdf['centroid_lon'] = centroids_wgs84.x
gdf['centroid_lat'] = centroids_wgs84.y

gdf.to_file('nyc_zip_codes_with_centroids.geojson', driver='GeoJSON')

print("GeoJSON with centroids saved as nyc_zip_codes_with_centroids.geojson")


GeoJSON with centroids saved as nyc_zip_codes_with_centroids.geojson
